In [167]:
from pandas import DataFrame, read_csv, to_datetime
import boto3
import os
from decimal import Decimal


In [168]:
import os
from pandas import DataFrame, read_csv

def load_electricity() -> DataFrame:
    current_dir = os.path.dirname(os.path.abspath('src'))
    file_path = os.path.join(current_dir, "data", "electricity_costs.csv")

    return DataFrame(read_csv(file_path))


def load_btc_prices() -> DataFrame:
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table("asics-dynamodb-PricesTable-1LXU143BUOBN")
    response = table.scan()
    items = response["Items"]
    df = DataFrame(items)
    df["Date"] = to_datetime(df["Date"])
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")
    df.sort_values(by="Date", inplace=True)
    return df


def load_difficulties() -> DataFrame:
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table("asics-dynamodb-DifficultyTable-DQ308ID3POT6")
    response = table.scan()
    items = response["Items"]
    df = DataFrame(items)
    df.sort_values(by="Date", inplace=True)
    return df

def load_miners() -> DataFrame:
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table("asics-dynamodb-MiningAsicSpecsTable-6NUBQKVF3PU5")
    response = table.scan()
    items = response["Items"]
    df = DataFrame(items)
    return df


In [169]:
electricity = load_electricity()
btc_prices = load_btc_prices()
difficulties = load_difficulties()
miners = load_miners()

In [170]:
merged_df = electricity.merge(btc_prices, on="Date")
merged_df = merged_df.merge(difficulties, on="Date")
merged_df = merged_df.merge(miners, how='cross')

merged_df["Date"] = to_datetime(merged_df["Date"])

In [171]:
import pandas as pd
from decimal import Decimal
from datetime import date

BTC_BLOCK_REWARDS = (
    (date(2009, 1, 3), Decimal(50)),
    (date(2012, 11, 28), Decimal(25)),
    (date(2016, 7, 9), Decimal(12.5)),
    (date(2020, 5, 11), Decimal(6.25)),
    (date(2024, 4, 20), Decimal(3.125)),
)

merged_df["Block Reward"] = merged_df["Date"].apply(lambda x: next((reward for date, reward in reversed(BTC_BLOCK_REWARDS) if date <= x), None))

/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/2984215447.py:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  merged_df["Block Reward"] = merged_df["Date"].apply(lambda x: next((reward for date, reward in reversed(BTC_BLOCK_REWARDS) if date <= x), None))


In [172]:
def make_decimals(data_frame, cols):
    data_frame[cols] = data_frame[cols].apply(lambda x: [Decimal(y) for y in x])

make_decimals(merged_df, ["Price (EUR/MWhe)", "Price", "Difficulty", "HashRate", "Power", "Block Reward"])

In [173]:
merged_df["Price (EUR/MWhe)"] = merged_df["Price (EUR/MWhe)"].abs()
merged_df["Electricity_day_price"] = merged_df["Price (EUR/MWhe)"] * 24 * merged_df["Power"] / Decimal(1000000)
merged_df[f'Electricity_day_price'] = merged_df[f'Electricity_day_price'].apply(lambda x: Decimal(x))

In [181]:
sorted_df = merged_df.sort_values(by=["Country", "AsicID", "Date"])


In [187]:
from decimal import Decimal

unique_countries = sorted_df["Country"].unique()

for country in unique_countries:
    for i in [2, 3, 4, 5, 6]:
        sorted_df.loc[sorted_df["Country"] == country, f'Cost_{i}_months'] = sorted_df['Electricity_day_price'].rolling(window=i*30).sum()
        sorted_df.loc[sorted_df["Country"] == country, f'Cost_{i}_months'] = sorted_df[f'Cost_{i}_months'].apply(lambda x: Decimal(x))

In [188]:
sorted_df = sorted_df.dropna()

sorted_df


,Country,ISO3 Code,Date,Price (EUR/MWhe),ID_x,Price,ID_y,Difficulty,AsicName,HashRate,Power,AsicID,Block Reward,Electricity_day_price,Cost_2_months,Cost_3_months,Cost_4_months,Cost_5_months,Cost_6_months
66976,Austria,AUT,2018-07-04,53.21999999999999886313162278,1744a31f-9b84-4f9f-8645-eb07e7419152,6597.5498046875,c2c3876d-0dc8-46cb-9bf2-c047e5c19114,5363678461481.4,Antminer S19j Pro 100TH/s,100,3050,03906dbe-f1bb-424b-a6ce-a328cf60567d,12.5,3.895703999999999916781234788,172.12906800000001794614945538341999053955078125,242.0240880000000061045284382998943328857421875,320.90147999999999228748492896556854248046875,409.50641999999999143255990929901599884033203125,481.4810519999999769424903206527233123779296875
67340,Austria,AUT,2018-07-05,51.67999999999999971578290570,9f443f70-7152-42fc-9df7-b7ae33b89e55,6639.14013671875,3c72284d-e565-49d0-af0f-e974438cfcd1,5363678461481.4,Antminer S19j Pro 100TH/s,100,3050,03906dbe-f1bb-424b-a6ce-a328cf60567d,12.5,3.782975999999999979195308698,174.504407999999983758243615739047527313232421875,243.300695999999987861883710138499736785888671875,321.14816400000000840009306557476520538330078125,409.9756320000000187064870260655879974365234375,482.60759999999999081410351209342479705810546875
67704,Austria,AUT,2018-07-06,47.74000000000000198951966013,08ae390e-2b56-4d6c-8dbe-60770f7edd96,6673.5,50d72fd4-b5e4-428e-b85a-1b4bb7ea7019,5363678461481.4,Antminer S19j Pro 100TH/s,100,3050,03906dbe-f1bb-424b-a6ce-a328cf60567d,12.5,3.494568000000000145632839121,175.449420000000003483364707790315151214599609375,245.56477200000000493673724122345447540283203125,321.9614159999999856154317967593669891357421875,410.02613999999999805368133820593357086181640625,484.3995360000000118816387839615345001220703125
68068,Austria,AUT,2018-07-07,32.28000000000000113686837722,bb87e8fe-7269-486a-8a3c-1fc8f4d23b30,6856.93017578125,ec26de8c-2597-4f83-8a16-d2eaa823c6d0,5363678461481.4,Antminer S19j Pro 100TH/s,100,3050,03906dbe-f1bb-424b-a6ce-a328cf60567d,12.5,2.362896000000000083218765213,175.803708000000000311047188006341457366943359375,246.155495999999999412466422654688358306884765625,321.55735199999998030762071721255779266357421875,408.9142319999999699575710110366344451904296875,484.54739999999998190105543471872806549072265625
68432,Austria,AUT,2018-07-08,34.63000000000000255795384874,868493d5-6631-4f4c-92b3-4b1049b5f3b6,6773.8798828125,2a54a5bb-bd3b-45be-aa8c-f5fb88d5d87e,5363678461481.4,Antminer S19j Pro 100TH/s,100,3050,03906dbe-f1bb-424b-a6ce-a328cf60567d,12.5,2.534916000000000187242221728,175.623636000000004742105375044047832489013671875,245.59039200000000846557668410241603851318359375,321.5500319999999874198692850768566131591796875,407.81842799999998305793269537389278411865234375,484.81531200000000580985215492546558380126953125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838484,Switzerland,CHE,2024-04-26,80.65000000000000568434188608,ccd44df7-c0ab-40e2-afc1-d4006d025a42,63755.3203125,4976e979-8a66-465d-a4a4-8f2d6223a0d7,86388558925171,Antminer S17 (56Th),56,2520,f3f66758-bae9-4a69-86a8-ac817f9d37d4,3.125,4.87771200000000034378899727,248.367167999999992389348335564136505126953125,376.72387200000002849265001714229583740234375,525.4556831999999531035427935421466827392578125,676.3478400000000192449078895151615142822265625,863.6634720000000697837094776332378387451171875
838861,Switzerland,CHE,2024-04-27,54.42000000000000170530256582,085dc676-50b5-448e-9f22-7ebdfec46bb9,63419.140625,8d041841-b918-4177-9e92-c39ef141af05,86388558925171,Antminer S17 (56Th),56,2520,f3f66758-bae9-4a69-86a8-ac817f9d37d4,3.125,3.291321600000000103136699182,247.704307199999988142735674045979976654052734375,375.17558400000001483931555412709712982177734375,527.8246847999999999956344254314899444580078125,669.9647808000000850370270200073719024658203125,862.9576703999999836014467291533946990966796875
839238,Switzerland,CHE,2024-04-28,24.32999999999999829469743418,ffccf465-cad8-4ed7-a11d-cd8c4da68f84,63113.23046875,2cf4574f-41b

In [189]:
def add_btc_needed(df: DataFrame) -> DataFrame:
    for i in range(2, 7):
        df[f"BTC_needed_{i}_months"] = df[f"Cost_{i}_months"] / df["Price"]
    return df

In [190]:
df = add_btc_needed(sorted_df)

/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/2797257916.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"BTC_needed_{i}_months"] = df[f"Cost_{i}_months"] / df["Price"]
/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/2797257916.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"BTC_needed_{i}_months"] = df[f"Cost_{i}_months"] / df["Price"]
/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/2797257916.py:3: SettingWithCopyWarning: 
A value is tr

In [191]:
df

,Country,ISO3 Code,Date,Price (EUR/MWhe),ID_x,Price,ID_y,Difficulty,AsicName,HashRate,...,Cost_2_months,Cost_3_months,Cost_4_months,Cost_5_months,Cost_6_months,BTC_needed_2_months,BTC_needed_3_months,BTC_needed_4_months,BTC_needed_5_months,BTC_needed_6_months
66976,Austria,AUT,2018-07-04,53.21999999999999886313162278,1744a31f-9b84-4f9f-8645-eb07e7419152,6597.5498046875,c2c3876d-0dc8-46cb-9bf2-c047e5c19114,5363678461481.4,Antminer S19j Pro 100TH/s,100,...,172.12906800000001794614945538341999053955078125,242.0240880000000061045284382998943328857421875,320.90147999999999228748492896556854248046875,409.50641999999999143255990929901599884033203125,481.4810519999999769424903206527233123779296875,0.02608984745787047457942365297,0.03668393497053164508590164063,0.04863949337252480717974647123,0.06206947004917633976435400558,0.07297876730811672011717034635
67340,Austria,AUT,2018-07-05,51.67999999999999971578290570,9f443f70-7152-42fc-9df7-b7ae33b89e55,6639.14013671875,3c72284d-e565-49d0-af0f-e974438cfcd1,5363678461481.4,Antminer S19j Pro 100TH/s,100,...,174.504407999999983758243615739047527313232421875,243.300695999999987861883710138499736785888671875,321.14816400000000840009306557476520538330078125,409.9756320000000187064870260655879974365234375,482.60759999999999081410351209342479705810546875,0.02628418807352437899804529270,0.03664641670302896222170985721,0.04837195139530831983853085078,0.06175131471206172191229564121,0.07269128080771590038532027586
67704,Austria,AUT,2018-07-06,47.74000000000000198951966013,08ae390e-2b56-4d6c-8dbe-60770f7edd96,6673.5,50d72fd4-b5e4-428e-b85a-1b4bb7ea7019,5363678461481.4,Antminer S19j Pro 100TH/s,100,...,175.449420000000003483364707790315151214599609375,245.56477200000000493673724122345447540283203125,321.9614159999999856154317967593669891357421875,410.02613999999999805368133820593357086181640625,484.3995360000000118816387839615345001220703125,0.02629046527309507806748553350,0.03679699887615194499688877519,0.04824476151944256920887567195,0.06144094403236682371374561148,0.07258553023151270126345078054
68068,Austria,AUT,2018-07-07,32.28000000000000113686837722,bb87e8fe-7269-486a-8a3c-1fc8f4d23b30,6856.93017578125,ec26de8c-2597-4f83-8a16-d2eaa823c6d0,5363678461481.4,Antminer S19j Pro 100TH/s,100,...,175.803708000000000311047188006341457366943359375,246.155495999999999412466422654688358306884765625,321.55735199999998030762071721255779266357421875,408.9142319999999699575710110366344451904296875,484.54739999999998190105543471872806549072265625,0.02563883596495419463906480912,0.03589878993801392614784691793,0.04689523500410489129133705535,0.05963517514649476350754468300,0.07066535425888198925495167094
68432,Austria,AUT,2018-07-08,34.63000000000000255795384874,868493d5-6631-4f4c-92b3-4b1049b5f3b6,6773.8798828125,2a54a5bb-bd3b-45be-aa8c-f5fb88d5d87e,5363678461481.4,Antminer S19j Pro 100TH/s,100,...,175.623636000000004742105375044047832489013671875,245.59039200000000846557668410241603851318359375,321.5500319999999874198692850768566131591796875,407.81842799999998305793269537389278411865234375,484.81531200000000580985215492546558380126953125,0.02592659436516040761119781307,0.03625549851026165803599484124,0.04746910744843221558885299849,0.06020455559520117596865762373,0.07157128859490578339524373720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838484,Switzerland,CHE,2024-04-26,80.65000000000000568434188608,ccd44df7-c0ab-40e2-afc1-d4006d025a42,63755.3203125,4976e979-8a66-465d-a4a4-8f2d6223a0d7,86388558925171,Antminer S17 (56Th),56,...,248.367167999999992389348335564136505126953125,376.72387200000002849265001714229583740234375,525.4556831999999531035427935421466827392578125,676.3478400000000192449078895151615142822265625,863.6634720000000697837094776332378387451171875,0.003895630463192961350386884005,0.005908900938046714930660713511,0.008241754266537313981180428150,0.01060849254124747707493384558,0.01354653176812082336416713423
838861,Switzerland,CHE,2024-04-27,54.420000000000